In [12]:
!pip install -q langchain langchain_community GitPython youtube-transcript-api sentence-transformers  chromadb openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.6/375.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 27.6 MB/s eta 0:00:00


In [17]:
from langchain_community.document_loaders import (
    GitLoader,
    YoutubeLoader,
    DataFrameLoader
)
from langchain.text_splitter import CharacterTextSplitter
import pandas as pd
import requests
from bs4 import BeautifulSoup

from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, Tool

In [3]:
# Knowledge base setup
kb_loader = GitLoader(
    clone_url="https://github.com/neo4j-documentation/knowledge-base",  # URL of the Git repository to clone
    repo_path="./repos/kb/",                                         # Local directory path where the repo will be cloned
    branch="master",                                                # The branch of the repository to check out
    file_filter=lambda file_path:                                   # A lambda function to filter which files to include
        file_path.endswith(".adoc") and                             # Include only files that end with '.adoc'
        "articles" in file_path                                      # AND the file path contains the substring 'articles'
)

# Load the data from the repository using the configured GitLoader
kb_data = kb_loader.load()

# Print the number of items (e.g., files or documents) loaded into kb_data
print(len(kb_data))


309


In [4]:
splitter = CharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=50,)

# GDS guides
gds_loader = GitLoader(
    clone_url="https://github.com/neo4j/graph-data-science",
    repo_path="./repos/gds/",
    branch="master",
    file_filter=lambda file_path: file_path.endswith(".adoc")
    and "pages" in file_path,
)
gds_data = gds_loader.load()

gds_data_chunks = splitter.split_documents(gds_data)
print(len(gds_data_chunks))

916


In [5]:
# Youtube
yt_loader = YoutubeLoader("1sRgsEKlUr0")
yt_data = yt_loader.load()
yt_data_split = splitter.split_documents(yt_data)
print(len(yt_data_split))

1


In [6]:
article_url = "https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/medium/neo4j_articles.csv"
medium = pd.read_csv(article_url, sep=";")
medium["source"] = medium["url"]
medium_loader = DataFrameLoader(medium[["text","source"]], page_content_column='text')
medium_data = medium_loader.load()
medium_data_chunks = splitter.split_documents(medium_data)
print(len(medium_data_chunks))

446


In [7]:
# Stackoverflow
so_data = []
for i in range(1, 20):
    # Define the Stack Overflow API endpoint and parameters
    api_url = "https://api.stackexchange.com/2.3/questions"
    params = {
        "order": "desc",
        "sort": "creation",
        "filter": "!-MBrU_IzpJ5H-AG6Bbzy.X-BYQe(2v-.J",
        "tagged": "neo4j",
        "site": "stackoverflow",
        "pagesize": 100,
        "page": i,
    }
    # Send GET request to Stack Overflow API
    response = requests.get(api_url, params=params)
    data = response.json()
    # Retrieve the resolved questions
    resolved_questions = [
        question
        for question in data["items"]
        if question["is_answered"] and question.get("accepted_answer_id")
    ]

    # Print the resolved questions
    for question in resolved_questions:
        text = (
            "Title:",
            question["title"] + "\n" + "Question:",
            BeautifulSoup(question["body"]).get_text()
            + "\n"
            + BeautifulSoup(
                [x["body"] for x in question["answers"] if x["is_accepted"]][0]
            ).get_text(),
        )
        source = question["link"]
        so_data.append(Document(page_content=str(text), metadata={"source": source}))
print(len(so_data))


808


In [11]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

sales_data = medium_data_chunks + yt_data_split
sales_store = Chroma.from_documents(
    sales_data,
    embeddings,
    collection_name="sales",
)

support_data = kb_data + gds_data_chunks + so_data
support_store = Chroma.from_documents(
    support_data,
    embeddings,
    collection_name="support",
)

In [13]:
from google.colab import userdata
open_ai_key = userdata.get('open_ai_key')

In [15]:
from openai import OpenAI
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(
    model_name="gpt-4",
    temperature=0,
    openai_api_key=open_ai_key,
    max_tokens=512,
)

<ipython-input-15-cb44648f894e>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [18]:
sales_template = """As a Neo4j marketing bot, your goal is to provide accurate and helpful information about Neo4j,
a powerful graph database used for building various applications.
You should answer user inquiries based on the context provided and avoid making up answers.
If you don't know the answer, simply state that you don't know.
Remember to provide relevant information about Neo4j's features, benefits,
and use cases to assist the user in understanding its value for application development.

{context}

Question: {question}"""
SALES_PROMPT = PromptTemplate(
    template=sales_template, input_variables=["context", "question"]
)

In [19]:
sales_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=sales_store.as_retriever(),
    chain_type_kwargs={"prompt": SALES_PROMPT},
    )

In [20]:
print(sales_qa.run("What are the main benefits of using Neo4j?"))

<ipython-input-20-d324fe17bdbd>:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(sales_qa.run("What are the main benefits of using Neo4j?"))


Neo4j is a powerful graph database that offers several benefits:

1. High Performance: Neo4j is designed to handle high-volume data and complex queries with ease. It can process large amounts of connected data faster than traditional relational databases.

2. Flexibility: Neo4j allows for flexible data modeling. It can easily adapt to changing business requirements without the need for complex joins or expensive schema updates.

3. Scalability: Neo4j can scale to handle large amounts of data and high user loads, making it suitable for enterprise-level applications.

4. Real-Time Insights: Neo4j's graph model allows for real-time querying and data analysis, providing immediate insights.

5. Easy Integration: Neo4j can be easily integrated with various programming languages and frameworks, making it a versatile choice for application development.

6. Rich Ecosystem: Neo4j has a rich ecosystem of tools, libraries, and community support, which can help developers build applications more ef

In [21]:
support_template = """
As a Neo4j Customer Support bot, you are here to assist with any issues
a user might be facing with their graph database implementation and Cypher statements.
Please provide as much detail as possible about the problem, how to solve it, and steps a user should take to fix it.
If the provided context doesn't provide enough information, you are allowed to use your knowledge and experience to offer you the best possible assistance.

{context}

Question: {question}"""

SUPPORT_PROMPT = PromptTemplate(
    template=support_template, input_variables=["context", "question"]
)

support_qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=support_store.as_retriever(),
    chain_type_kwargs={"prompt": SUPPORT_PROMPT},
)


In [22]:
print(support_qa.run("""
I am having my graph in a VM and i want to use GDS Plugins in my graph.
I didn't see any proper documentation to install it in my server where it was working only on locally.
Anyone clear me this to install the GDS Plugin in the VM ?
"""))

To install the Graph Data Science (GDS) library on a Neo4j server running on a Virtual Machine (VM), you can follow these steps:

1. Download the GDS library from the Neo4j Download Center. Make sure to choose the version that matches your Neo4j version.

2. Copy the downloaded .jar file to the plugins directory of your Neo4j server. The default location of the plugins directory is `<NEO4J_HOME>/plugins`. If you can't find the plugins directory, you may need to create it.

3. Add the following line to your Neo4j configuration file (`<NEO4J_HOME>/conf/neo4j.conf`):

```
dbms.security.procedures.unrestricted=gds.*
```

4. Restart your Neo4j server.

5. To verify the installation, run the following command in the Neo4j Browser:

```
RETURN gds.version()
```

If the GDS library is installed correctly, this command will return the version of the GDS library.

Please note that `<NEO4J_HOME>` refers to the directory where Neo4j is installed. The exact path depends on your specific installatio

In [23]:
tools = [
    Tool(
        name="sales",
        func=sales_qa.run,
        description="""useful for when a user is interested in various Neo4j information,
                       use-cases, or applications. A user is not asking for any debugging, but is only
                       interested in general advice for integrating and using Neo4j.
                       Input should be a fully formed question."""
    ),
     Tool(
        name = "support",
        func=support_qa.run,
        description="""useful for when when a user asks to optimize or debug a Cypher statement or needs
                       specific instructions how to accomplish a specified task.
                       Input should be a fully formed question."""
    ),
]

In [24]:
agent = initialize_agent(
    tools,
    llm,
    agent="zero-shot-react-description",
    verbose=True
)

<ipython-input-24-6b26c4f4fa2a>:1: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  agent = initialize_agent(


In [25]:
agent.run("""What are some GPT-4 applications with Neo4j?""")

The user is asking for general advice about the applications of GPT-4 with Neo4j. This is not a debugging or optimization question, but rather a request for information.
Action: sales
Action Input: What are some GPT-4 applications with Neo4j?
Observation: As an AI, I can't predict future developments such as GPT-4. However, based on the capabilities of GPT-3 and the features of Neo4j, potential applications could include:

1. Enhanced Natural Language Processing: GPT-4 could potentially offer more advanced natural language processing capabilities. This could be used in conjunction with Neo4j to create more sophisticated chatbots or virtual assistants that can understand and respond to user queries more effectively.

2. Advanced Data Analysis: GPT-4 could be used to analyze and interpret the data stored in a Neo4j graph database. This could be used for predictive analytics, trend analysis, or to gain insights from complex data sets.

3. Improved Recommendation Systems: GPT-4 could be us

'Potential applications of GPT-4 with Neo4j could include enhanced natural language processing, advanced data analysis, improved recommendation systems, enhanced data visualization, and improved semantic search. These are speculative applications based on the capabilities of GPT-3 and the features of Neo4j. The actual capabilities and applications of GPT-4 will depend on the advancements made in its development.'

In [26]:
agent.run("""
Hello everyone, is there a way to execute a weighted shortest path query in the Neo4j Community edition?
All I have found on the internet was gds library or the algo library, which are both unavailable the community version.
""")

The user is asking for specific instructions on how to accomplish a task in Neo4j Community edition. This falls under the support category.

Action: support

Action Input: 
Hello everyone, is there a way to execute a weighted shortest path query in the Neo4j Community edition? All I have found on the internet was gds library or the algo library, which are both unavailable the community version.

Observation: Yes, you can execute a weighted shortest path query in Neo4j Community Edition using Cypher's built-in shortestPath function. However, please note that this function only considers the number of hops (relationships) between nodes, and does not take into account any weights on these relationships.

If you need to consider weights, you can use APOC Procedures, which is a library of user-defined procedures and functions that works with all editions of Neo4j, including the Community Edition. The APOC path-finding procedures include support for weighted shortest path.

Here is an exampl

"Yes, you can execute a weighted shortest path query in Neo4j Community Edition using Cypher's built-in shortestPath function. However, this function only considers the number of hops (relationships) between nodes, and does not take into account any weights on these relationships.\n\nIf you need to consider weights, you can use APOC Procedures, which is a library of user-defined procedures and functions that works with all editions of Neo4j, including the Community Edition. The APOC path-finding procedures include support for weighted shortest path.\n\nHere is an example of how you can use APOC to find the weighted shortest path:\n\n```cypher\nMATCH (start:Node {name: 'Start'}), (end:Node {name: 'End'})\nCALL apoc.algo.dijkstra(start, end, 'RELATIONSHIP_TYPE', 'weight') YIELD path, weight\nRETURN path, weight\n```\n\nIn this query, 'RELATIONSHIP_TYPE' is the type of the relationships between nodes, and 'weight' is the property on the relationships that represents the weight. The query 